# First version of recommendation model

### Links

Tutorial: [link](https://www.stepbystepdatascience.com/hybrid-recommender-lightfm-python)

Dataset: [instacart-market-basket-analysis](https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis)

### Prerequisites

Download dataset and place files in folder: ../data/instacart-market-basket-analysis

In [1]:
# %pip install --upgrade pip setuptools wheel
# %pip install numpy pandas scikit-learn optuna nbformat
# %pip install --no-use-pep517 lightfm # https://github.com/lyst/lightfm/issues/687#issuecomment-1523956355
# ! pip install ipywidgets --upgrade


In [2]:
import pickle
import sys
from os import path

import numpy as np
import pandas as pd
import optuna
import sklearn

from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split

sys.path.insert(0, '..') # важно чтобы работали импорты из корня репозитория как в инференсе
sklearn.show_versions()


System:
    python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
executable: /home/vorkov/Workspace/recommendation/.venv/bin/python
   machine: Linux-6.5.0-21-generic-x86_64-with-glibc2.35

Python dependencies:
      sklearn: 1.4.1.post1
          pip: 24.0
   setuptools: 69.5.1
        numpy: 1.26.4
        scipy: 1.12.0
       Cython: None
       pandas: 2.2.2
   matplotlib: None
       joblib: 1.3.2
threadpoolctl: 3.4.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: openblas
    num_threads: 16
         prefix: libopenblas
       filepath: /home/vorkov/Workspace/recommendation/.venv/lib/python3.10/site-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
        version: 0.3.23.dev
threading_layer: pthreads
   architecture: Cooperlake

       user_api: openmp
   internal_api: openmp
    num_threads: 16
         prefix: libgomp
       filepath: /home/vorkov/Workspace/recommendation/.venv/lib/python3.10/site-packages/scikit_learn.lib

In [3]:
data_path = '../data'
dataset_path = path.join(data_path, 'instacart-market-basket-analysis')

# Маппинг user_id <-> order_id
raw_orders = pd.read_csv(path.join(dataset_path, 'orders.csv'))

# Продукты
raw_products = pd.read_csv(path.join(dataset_path, 'products.csv'))

# Маппинг order_id -> product_id
raw_order_products = pd.concat([pd.read_csv(path.join(dataset_path, 'order_products__prior.csv')),
                            pd.read_csv(path.join(dataset_path, 'order_products__train.csv'))])

# Фичи продуктов
aisles = pd.read_csv(path.join(dataset_path, 'aisles.csv'))
departments = pd.read_csv(path.join(dataset_path, 'departments.csv'))

# Чеки
raw_orders[(raw_orders["user_id"]==1)].sort_values(["order_number"])

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [4]:
# Количество юзеров
raw_orders['user_id'].unique().size

206209

In [5]:
# Айтемы
raw_products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [6]:
# Взаимодействия
raw_order_products

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [7]:
# Подрежем количество продуктов, возьмем пока топ N продуктов
top_products_count = 14
top_popular_product_counts = raw_order_products.groupby('product_id').size().sort_values()[-top_products_count:].rename('count').to_frame().reset_index()
top_popular_product_counts

,product_id,count
0,45007,109412
1,24964,113936
2,22935,117716
3,27966,142603
4,27845,142813
5,26209,146660
6,16797,149445
7,47626,160792
8,47766,184224
9,47209,220877


In [8]:
products = top_popular_product_counts.merge(raw_products, how='left', on='product_id')[['product_id', 'product_name']]
# products[['product_id', 'aisle_id']].merge(aisles, how='left', on='aisle_id')[['product_id', 'aisle']]
# products.to_csv(path.join(dataset_path, 'used_products.csv'), index=False)
products

,product_id,product_name
0,45007,Organic Zucchini
1,24964,Organic Garlic
2,22935,Organic Yellow Onion
3,27966,Organic Raspberries
4,27845,Organic Whole Milk
5,26209,Limes
6,16797,Strawberries
7,47626,Large Lemon
8,47766,Organic Avocado
9,47209,Organic Hass Avocado


In [9]:
order_products = products[['product_id']].merge(raw_order_products, how='left', on='product_id')
order_products

,product_id,order_id,add_to_cart_order,reordered
0,45007,131,6,1
1,45007,162,6,1
2,45007,306,3,1
3,45007,317,3,0
4,45007,335,8,0
...,...,...,...,...
2901976,24852,3419531,2,1
2901977,24852,3419542,6,0
2901978,24852,3419629,5,1
2901979,24852,3420088,9,1


In [10]:
orders = raw_orders[raw_orders['order_id'].isin(order_products['order_id'].unique())]
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1,2398795,1,prior,2,3,7,15.0
4,431534,1,prior,5,4,15,28.0
10,1187899,1,train,11,4,8,14.0
11,2168274,2,prior,1,2,11,NaN
12,1501582,2,prior,2,5,10,10.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [11]:
user_ids = orders["user_id"].unique()
product_ids = products["product_id"].unique()

dataset = Dataset()
dataset.fit(user_ids, product_ids)

user_mappings = dataset.mapping()[0]
item_mappings = dataset.mapping()[2]

len(user_mappings), len(item_mappings), list(user_mappings.items())[:5]

(169477, 14, [(1, 0), (2, 1), (3, 2), (6, 3), (7, 4)])

In [12]:
# Обратная матрица взаимодействий
inv_user_mappings = {v:k for k, v in user_mappings.items()}
inv_item_mappings = {v:k for k, v in item_mappings.items()}
list(inv_item_mappings.items())[:5]

[(0, 45007), (1, 24964), (2, 22935), (3, 27966), (4, 27845)]

In [13]:
# Создаем матрицу взаимодействий
data = orders.merge(order_products, how='inner', on='order_id').groupby(['user_id', 'product_id']).size()
data = data.to_frame().reset_index().rename(columns={0: 'weight'})
interactions, weights = dataset.build_interactions(data[['user_id', 'product_id', 'weight']].values)
interactions, weights

(<169477x14 sparse matrix of type '<class 'numpy.int32'>'
 	with 651269 stored elements in COOrdinate format>,
 <169477x14 sparse matrix of type '<class 'numpy.float32'>'
 	with 651269 stored elements in COOrdinate format>)

In [14]:
interactions.todense(), weights.todense()

(matrix([[0, 0, 0, ..., 0, 1, 0],
         [0, 0, 0, ..., 0, 1, 1],
         [0, 0, 0, ..., 1, 0, 0],
         ...,
         [1, 0, 0, ..., 1, 1, 0],
         [1, 1, 1, ..., 1, 1, 0],
         [0, 0, 0, ..., 0, 1, 1]], dtype=int32),
 matrix([[ 0.,  0.,  0., ...,  0.,  2.,  0.],
         [ 0.,  0.,  0., ...,  0.,  1.,  8.],
         [ 0.,  0.,  0., ...,  1.,  0.,  0.],
         ...,
         [ 5.,  0.,  0., ...,  1.,  8.,  0.],
         [ 5.,  5.,  7., ..., 15., 22.,  0.],
         [ 0.,  0.,  0., ...,  0.,  6.,  5.]], dtype=float32))

In [15]:
def evaluate_model(model):
    train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=42)

    for metric in [precision_at_k, recall_at_k]:        
        print(f"{metric.__name__}:")
        print(f"\t- train: %.3f" % metric(model, train, k=5).mean())
        print(f"\t- test: %.3f" % metric(model, test, k=5).mean())
        print(f"\t- test new: %.3f" % metric(model, test, train, k=5).mean())

In [16]:
def objective(trial):

    train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=42)

    param = {
        'no_components': trial.suggest_int("no_components", 5, 64),
        "learning_schedule": trial.suggest_categorical("learning_schedule", ["adagrad", "adadelta"]),
        "loss":  trial.suggest_categorical("loss", ["bpr", "warp", "warp-kos"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 1),
        "item_alpha": trial.suggest_float("item_alpha", 1e-10, 1e-06, log=True),
        "user_alpha": trial.suggest_float("user_alpha", 1e-10, 1e-06, log=True), 
        "max_sampled": trial.suggest_int("max_sampled", 5, 15),
    }
    epochs = trial.suggest_int("epochs", 20, 50)

    model = LightFM(**param, random_state=123)
    model.fit(train, epochs=epochs, verbose=True)

    val_precision = precision_at_k(model, test, k=5).mean()

    return val_precision

if True:
    best_params = {
        'no_components': 5,
        'learning_schedule': 'adagrad',
        'loss': 'warp',
        'learning_rate': 0.7510216830421954,
        'item_alpha': 6.201256477126263e-09,
        'user_alpha': 9.592173442318563e-09,
        'max_sampled': 12,
        'epochs': 46,
    }
    print("No learning")
    display(best_params)
else:
    study = optuna.create_study(direction="maximize")
    
    study.enqueue_trial(params={
        "no_components": 10, 
        "learning_schedule": 'adagrad', 
        "loss": 'warp',
        "learning_rate": 0.5,
        "item_alpha": 1e-9, 
        "user_alpha": 1e-8, 
        "max_sampled": 15,
        "epochs": 30,
    })
    
    study.optimize(objective, n_trials=50)
    
    best_params = study.best_params
    for k, v in best_params.items():
        print(k, ":", v)
    
    display(optuna.importance.get_param_importances(study))

No learning


{'no_components': 5,
 'learning_schedule': 'adagrad',
 'loss': 'warp',
 'learning_rate': 0.7510216830421954,
 'item_alpha': 6.201256477126263e-09,
 'user_alpha': 9.592173442318563e-09,
 'max_sampled': 12,
 'epochs': 46}

In [17]:
if 'epochs' in best_params:
    num_epochs = best_params['epochs']
    del best_params['epochs']

model = LightFM(**best_params, random_state=123)

model.fit(
    interactions,
    epochs=num_epochs,
    verbose=True
)

evaluate_model(model)

Epoch: 100%|██████████| 46/46 [00:23<00:00,  1.93it/s]


precision_at_k:
	- train: 0.503
	- test: 0.234
	- test new: 0.293
recall_at_k:
	- train: 0.874
	- test: 0.819
	- test new: 0.980


In [18]:
# Сравним с топ 5 рекомендацией
# todo сделать подсчет метрики для новых пользователей

k = 5
static_prediction = top_popular_product_counts[-k:]['product_id']
static_prediction_ind = list(map(lambda x: item_mappings[x], static_prediction))

def static_precision_at_k(interactions, suppress_interactions=None):
    interactions = interactions.copy()
    interactions.data = np.isin(interactions.col, static_prediction_ind)
    hit = np.squeeze(np.array(interactions.sum(axis=1)))
    return hit / k

def static_recall_at_k(interactions, suppress_interactions=None):
    retrieved = np.squeeze(interactions.getnnz(axis=1))

    interactions = interactions.copy()
    interactions.data = np.isin(interactions.col, static_prediction_ind)
    hit = np.squeeze(np.array(interactions.sum(axis=1)))

    hit = hit[retrieved > 0]
    retrieved = retrieved[retrieved > 0]

    return hit / retrieved


train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=42)
for metric in [static_precision_at_k, static_recall_at_k]:        
    print(f"{metric.__name__}:")
    print(f"\t- train: %.3f" % metric(train).mean())
    print(f"\t- test: %.3f" % metric(test).mean())
    # print(f"\t- test new: %.3f" % metric(test, train).mean())

static_precision_at_k:
	- train: 0.287
	- test: 0.072
static_recall_at_k:
	- train: 0.510
	- test: 0.484


In [21]:
# Пример инференса
from inference.model import Model

recs_model = Model(model, dataset)
recs_product_ids = recs_model.get_recommendations(user_ids=1)
recs_product_ids

['Зеленые яблоки', 'Лимон', 'Авокадо', 'Замороженый шпинат', 'Молоко']

In [22]:
%%timeit
recs_model.get_recommendations(1)

66.2 µs ± 8.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [23]:
recs = products[products['product_id'].isin(recs_product_ids)]
recs

,product_id,product_name


In [24]:
# Сохраним модель
with open('../recommendation/0-0-1.pickle', 'wb') as file:
    pickle.dump(recs_model, file, protocol=pickle.HIGHEST_PROTOCOL)